In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback, allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms, set_joint_vector_field, show_center_joint_normal, show_joint_normal

from design_optimization_analysis import DesignOptimizationAnalysis

import umbrella_optimization
import umbrella_optimization_finite_diff
from umbrella_optimization import OptEnergyType

import numpy as np
import numpy.linalg as la

import pickle, gzip

from configuration import *

import parallelism
parallelism.set_max_num_tbb_threads(24)
parallelism.set_hessian_assembly_num_threads(8)
parallelism.set_gradient_assembly_num_threads(8)

### Initialization

In [ ]:
name = 'hive'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=True, handlePivots = True)

#### Pin Rigid Motion

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

rod_colors = get_color_field(curr_um, input_data)

### Load Optimized Model

### Initialize Design Optimization

In [ ]:
import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False

### Load TSF optimized model

In [ ]:
curr_um = pickle.load(gzip.open("2022_05_02_11_35_hive/hive_", 'r'))

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
# optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

# optimizer.beta = 1 * 1e6
# optimizer.gamma = 1
# optimizer.eta = 0
# optimizer.zeta = 0# 1e1
# optimizer.iota = 0

# optimizer.objective.terms[-1].term.normalActivationThreshold = -1e-5

optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1
optimizer.iota = 1e9

optimizer.objective.terms[-1].term.normalActivationThreshold = -1e-6
optimizer.objective.terms[-1].term.normalWeight = 1
optimizer.objective.terms[-1].term.tangentialWeight = 0
optimizer.objective.terms[-1].term.torqueWeight = 0

In [ ]:
allDesignObjectives(optimizer)

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
camParams = view.getCameraParams()

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
configure_umbrella_true_equlibrium(curr_um, thickness, target_height_multiplier)
curr_um.attractionWeight = 1e-16
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success

### Load force optimized model

In [ ]:
curr_um = pickle.load(gzip.open("2022_05_02_11_08_lilium_smooth/lilium_smooth_tsf+force_equilibrium_2022_05_02_11_08_target_height_factor_5.0.pkl.gz", 'r'))

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1
optimizer.iota = 1e9

optimizer.objective.terms[-1].term.normalActivationThreshold = -1e-6
optimizer.objective.terms[-1].term.normalWeight = 1
optimizer.objective.terms[-1].term.tangentialWeight = 0
optimizer.objective.terms[-1].term.torqueWeight = 0

In [ ]:
allDesignObjectives(optimizer)

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
curr_um.attractionWeight = 1e-7
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success

In [ ]:
view.setCameraParams(camParams)

In [ ]:
import force_analysis
with so(): importlib.reload(force_analysis)

In [ ]:
force_analysis.UmbrellaForceAnalysis(curr_um)
v1 = force_analysis.UmbrellaForceFieldVisualization(curr_um)
v1.show()

### Visualize the ablation result

In [ ]:
file_name = "2022_05_02_00_38_hemisphere_5t/hemisphere_5t_log_hemisphere_5t.txt"

In [ ]:
import matplotlib.pyplot as plt
import re
# Prepare table
columns = ('Objective Types', 'Elastic Energy', 'TSF', 'Forces')
rows = ["TSF only", "TSF + Energy", "TSF + Energy + Forces", "TSF + Forces"]
highlight = "#D9E76C"
colors = [["w", "w", highlight ,"w"],
          ["w", highlight, highlight ,"w"],
          ["w", highlight, highlight ,highlight], 
          ["w", "w", highlight ,highlight]]


cell_text = []

with open(file_name, 'r') as f:
    content = f.readlines()
    row_counter = 0
    for line in content:
        if "Optimization objective" in line:
            values = re.split(": |, ", line)
            row_value = [rows[row_counter], np.round(float(values[4]), 6), np.round(float(values[8]), 6), np.round(float(values[10]), 6)]
            cell_text.append(row_value)
            row_counter += 1

fig, ax = plt.subplots()
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=cell_text,cellColours=colors,
                     colLabels=columns,loc='center')
table.auto_set_font_size(False)
table.set_fontsize(18)
table.scale(4, 4)
plt.show()

In [ ]:
initial_umbrella = pickle.load(gzip.open("2022_05_01_23_50_lilium_smooth_new_10t/lilium_smooth_new_10t_tsf_only_equilibrium_2022_05_01_23_50_target_height_factor_10.0.pkl.gz"))

In [ ]:
import importlib
import stress_analysis
importlib.reload(stress_analysis)
stress_vis = stress_analysis.stressFieldVisualization(initial_umbrella, target_mesh)
stress_vis.showInitial()

In [ ]:
optimView1 = stress_vis.getView(curr_um)
from ipywidgets import HBox
HBox([stress_vis.initialView.show(), optimView1.show()])

In [ ]:
second_umbrella = pickle.load(gzip.open("2022_05_01_23_50_lilium_smooth_new_10t/lilium_smooth_new_10t_tsf+energy_equilibrium_2022_05_01_23_50_target_height_factor_10.0.pkl.gz"))

In [ ]:
importlib.reload(stress_analysis)
stress_analysis.plotStressDistributions([initial_umbrella, curr_um, second_umbrella], ['Initial_Deployment', 'Optimized with zeta = 1e3', "tsf + energy"], stress_vis.vmin, stress_vis.vmax)

### Continue the optimization

In [ ]:
curr_um.attractionWeight

In [ ]:
import time
### Optimization
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
#algorithm = umbrella_optimization.OptAlgorithm.BFGS
arm_length_lower_bound = input_data["plate_edge_length"] / 30 * 32


In [ ]:
opt_opts.niter = 50

optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
rest_height_optimizer.newPt(rest_height_optimizer.params())

rest_height_optimizer.beta = 1e6
rest_height_optimizer.gamma = 1
rest_height_optimizer.eta = 0
rest_height_optimizer.zeta = 0# 1e1
rest_height_optimizer.iota = 0


doptays = DesignOptimizationAnalysis(rest_height_optimizer)

pipeline_helper.prev_time_stamp = time.time()

uo = rest_height_optimizer.get_parent_opt()

uo.equilibriumOptimizer.options.verbose = 1
#uo.equilibriumOptimizer.options.verboseWorkingSet = True
uo.equilibriumOptimizer.options.gradTol = 1e-10
# Hold the closest points fixed in the target-attraction term of the equilibrium solve:
# this seems to make the design optimization much more robust.
uo.setHoldClosestPointsFixed(True, False)

tfview = pipeline_helper.TargetFittingVisualization(curr_um, uo.target_surface_fitter, view)
cb = pipeline_helper.UmbrellaOptimizationCallback(rest_height_optimizer, view, True, False, 1, rod_colors, doptays.record, tfview=tfview)

solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 500, 0.005, 1e-5, cb, arm_length_lower_bound)

In [ ]:
rest_height_optimizer.iota = 1e10

In [ ]:
allDesignObjectives(rest_height_optimizer)

In [ ]:
pickle.dump(curr_um, gzip.open("2022_05_01_16_11_hive/hive_tsf+energy_after_all_term_equilibrium_2022_05_01_16_11_target_height_factor_5.0.pkl.gz", 'w'))

In [ ]:
with open("2022_05_01_16_11_hive/hive_tsf+energy_after_all_term_log.txt", "w") as f:
    f.write("Optimization objective with constant weight: {}\n".format(allDesignObjectives(rest_height_optimizer)))